# Гипотеза о разнице в `font_family`

Предполагается следующее: если у нас есть хорошо предсказывающая ширину текстов модель `M`, работающая для шрифта `F`, то достаточно хорошую модель для шрифта `G` можно получить из `M` простым домножением ее предсказаний на некоторую константу `c(F, G)`, отвечающую за переход от шрифта `F` к шрифту `G`.

### Подготовка

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

import os; import sys; sys.path.append(os.path.join(sys.path[0], ".."))
from util import util

In [2]:
CARTS_COUNT = 10
BASIC_FONT = util.Font("Lucida Grande", 20, "italic")
MONOSPACED_FONT_FAMILIES = [
    "Courier",
    "Lucida Console"
]

In [3]:
char_widths_df = util.get_df("../data/full/char_widths.csv", "all")
char_widths_df = char_widths_df[~char_widths_df.font_family.isin(MONOSPACED_FONT_FAMILIES)]
char_widths_df = util.filter_by_font(char_widths_df, BASIC_FONT, filters=["size", "face"])
char_widths_df

,char_id,char,alphabet,font_family,width
0,65,A,basic_latin,Geneva,19
1,66,B,basic_latin,Geneva,18
2,67,C,basic_latin,Geneva,21
3,68,D,basic_latin,Geneva,20
4,69,E,basic_latin,Geneva,19
...,...,...,...,...,...
1409,123,{,russian,Arial,13
1410,124,|,russian,Arial,7
1411,125,},russian,Arial,11
1412,126,~,russian,Arial,16


In [4]:
char_widths_s = char_widths_df.groupby(["font_family", "char"]).width.median().astype(int)
char_widths_s

font_family  char
Arial                 8
             !        9
             "       12
             #       16
             $       16
                     ..
Verdana      ы       21
             ь       15
             э       16
             ю       23
             я       17
Name: width, Length: 1113, dtype: int32

## Шаг №1

Разбиение символов по корзинам.

### Шаг №1.1

Ширины символов пробразуются в "порядки": для каждого символа `X` средний порядок `p(X)` - это (медианное по всем шрифтам `F`) количество символов, которые по ширине меньше, чем `X`.

In [5]:
mean_order_df = util.transform_values_to_orders(
    char_widths_df[char_widths_df.font_family == BASIC_FONT.family],
    values_col="width",
    sorted_col="char"
).T.reset_index()
mean_order_df

,char,order
0,|,0
1,',0
2,,2
3,i,2
4,I,2
...,...,...
154,Ш,154
155,@,154
156,Ю,156
157,Ж,157


### Шаг №1.2

Полученные значения `p(X)` складываем ровно в `CARTS_COUNT` корзин.

In [6]:
mean_order_df["cart_id"] = KMeans(n_clusters=CARTS_COUNT, random_state=42).fit(mean_order_df[["order"]]).labels_
cart_id_replaces = {
    k: i
    for i, k in enumerate(mean_order_df.groupby("cart_id")["order"].median().sort_values().keys())
}
mean_order_df = mean_order_df.replace({"cart_id": cart_id_replaces})
mean_order_df

,char,order,cart_id
0,|,0,0
1,',0,0
2,,2,0
3,i,2,0
4,I,2,0
...,...,...,...
154,Ш,154,9
155,@,154,9
156,Ю,156,9
157,Ж,157,9


Рисуем `geom_boxplot()` с вариацией `p(X)` для каждой корзины:

In [7]:
ggplot(mean_order_df, aes(as_discrete("cart_id", order_by="..middle.."), "order")) + geom_boxplot()

### Шаг №1.3

Фиксируем следующие значения:

- `basic_cart_id` - индекс средней по ширине корзины.

- `BASIC_FONT` - "Lucida Grande", 14pt, без модификаций.

- `cart_widths` - словарь, в котором каждая корзина получает в качестве ширины медианное значение ширин (в пикселях) всех символов корзины, начертанных базовым шрифтом.

- `basic_width` - значение из `cart_widths`, принадлежащее корзине с индексом `basic_cart_id`.

In [8]:
cart_widths_df = pd.merge(char_widths_s.to_frame().reset_index(), mean_order_df, on="char").groupby(["font_family", "cart_id"]).width.median().reset_index(level=1)
cart_widths_df

,cart_id,width
font_family,,
Arial,0,8.0
Arial,1,12.0
Arial,2,15.0
Arial,3,16.0
Arial,4,18.0
...,...,...
Verdana,5,19.0
Verdana,6,21.0
Verdana,7,21.0


In [9]:
basic_cart_id = int(CARTS_COUNT / 2)
print("Basic cart_id:", basic_cart_id)
print("Basic font:", BASIC_FONT)
cart_widths = cart_widths_df.loc[BASIC_FONT.family].set_index("cart_id").width.astype(int).to_dict()
print("Cart widths:", cart_widths)
basic_width = cart_widths[basic_cart_id]
print("Basic width:", basic_width)

Basic cart_id: 5
Basic font: Font(family='Lucida Grande', size=20, face='italic')
Cart widths: {0: 8, 1: 12, 2: 15, 3: 16, 4: 18, 5: 19, 6: 20, 7: 21, 8: 22, 9: 25}
Basic width: 19


#### Сравнение медианных ширин кластеров (базовый шрифт vs. текущий).

In [10]:
util.plot_matrix([
    ggplot(mapping=aes("cart_id", "width", color="font_family")) + \
        geom_line(data=cart_widths_df.loc[BASIC_FONT.family].reset_index()) + \
        geom_point(data=cart_widths_df.loc[BASIC_FONT.family].reset_index()) + \
        geom_line(data=cart_widths_df.loc[family].reset_index()) + \
        geom_point(data=cart_widths_df.loc[family].reset_index()) + \
        scale_x_continuous(breaks=list(cart_widths.keys()))
    for family in (set(cart_widths_df.index) - set([BASIC_FONT.family]))
], width=480, height=240)

### Шаг №1.4

Вычисляются нормировочные коэффициенты различных шрифтов. Для каждого `F` его нормировочный коэффициент `N(F)` - это отношение медианной ширины "базовой корзины" начертанной шрифтом `F` к `basic_width`.

In [11]:
ncoeff_s = char_widths_s.to_frame().reset_index().set_index("char").loc[mean_order_df[mean_order_df.cart_id == basic_cart_id].char.values].reset_index().groupby(["font_family"]).width.median() / basic_width
ncoeff_s

font_family
Arial              1.000000
Geneva             1.000000
Georgia            1.052632
Helvetica          1.000000
Lucida Grande      1.000000
Times New Roman    1.000000
Verdana            1.000000
Name: width, dtype: float64

### Шаг №1.5

С помощью нормировочных коэффициентов восстанавливаются прогнозируемые ширины символов в пикселях: для каждого символа `X` и шрифта `F` шириной становится ширина корзины которой принадлежит символ, умноженная на нормировочный коэффициент данного шрифта.

In [12]:
char_widths_calc_df = char_widths_s.to_frame().reset_index()\
    .merge(mean_order_df, on="char")\
    .merge(ncoeff_s.to_frame().reset_index(), \
           on=["font_family"], \
           suffixes=("_original", "_ncoeff"))\
    .assign(width_calc=lambda r: (np.round(r.cart_id.map(cart_widths) * r.width_ncoeff)).astype(int))
char_widths_calc_df.head()

,font_family,char,width_original,order,cart_id,width_ncoeff,width_calc
0,Arial,,8,2,0,1.0,8
1,Arial,!,9,11,1,1.0,12
2,Arial,"""",12,19,1,1.0,12
3,Arial,#,16,63,3,1.0,16
4,Arial,$,16,63,3,1.0,16


### Шаг №1.6

Второй `geom_boxplot()` с вариацией спрогнозированных ширин символов для каждой корзины.

In [13]:
ggplot(char_widths_calc_df, aes(as_discrete("cart_id", order=-1), "width_calc")) + geom_boxplot()

## Шаг №2

Другие способы проверить гипотезу.

In [14]:
def add_basic_width(df, basic_family):
    def calc_basic_width(r):
        basic_width = int(r[r.font_family == basic_family].iloc[0].width)
        r["basic_width"] = pd.Series(basic_width, index=r.index)
        return r

    df["basic_width"] = np.nan
    return df.groupby("text").apply(calc_basic_width)

In [15]:
control_df = util.get_df("../data/full/control.csv", "all")
control_df = util.filter_by_font(control_df, BASIC_FONT, filters=["size", "face"])
control_df.head()

,text,width,alphabet,font_family,symbols_count
0,Да.,57,russian,Courier,3.0
1,Аж.,57,russian,Courier,3.0
2,Аж.,57,russian,Courier,3.0
3,Миг.,72,russian,Courier,4.0
4,При.,72,russian,Courier,4.0


In [16]:
control_df = add_basic_width(control_df, BASIC_FONT.family)
control_df["font_coeff"] = control_df.width / control_df.basic_width
control_df.head()

,text,width,alphabet,font_family,symbols_count,basic_width,font_coeff
0,Да.,57,russian,Courier,3.0,44,1.295455
1,Аж.,57,russian,Courier,3.0,46,1.239130
2,Аж.,57,russian,Courier,3.0,46,1.239130
3,Миг.,72,russian,Courier,4.0,58,1.241379
4,При.,72,russian,Courier,4.0,58,1.241379


Теперь для каждого текста `text` и шрифта `F` величина `font_coeff(text, F)` - это отношение ширины этого текста написанного шрифтом `F` к ширине этого текста написанного базовым шрифтом.

По идее, если ширину для одного шрифта можно получить из ширины для другого, домноженной на коэффициент шрифта, то `font_coeff(text, F)` должен оставаться постоянным при переборе текстов `text` при постоянном `F`. Но график ниже показывает, что это не всегда так.

Это значит, что ошибка будет иметь место, если мы будем пытаться получить модель для шрифта `F1` из модели для шрифта `F2` простым домножением на подходящую константу. С одной стороны, эта ошибка не всегда так уж велика. С другой стороны, она тем больше, чем короче текст - и это в ситуации "идеальной модели", когда мы хотя бы для одного шрифта умеем абсолютно правильно вычислять ширину любого текста, но для более простых моделей данный эффект лишь вносит дополнительное искажение.

Видно также, что эти графики согласуются с предыдущими. `font_coeff(text, F)` константен для тех же шрифтов, для которых сравнение медианных ширин кластеров давало кривые, полностью совпадающие с базовой.

In [17]:
plot_df = control_df.groupby(["alphabet", "font_family", "symbols_count"])\
                    .font_coeff.agg(["min", "max", "median"]).reset_index()
ggplot(plot_df[plot_df.alphabet == "basic_latin"]) + \
    geom_pointrange(aes(x="symbols_count", y="median", ymin="min", ymax="max")) + \
    facet_grid(y="font_family") + \
    ggtitle("Вариация font_coeff в зависимости от количества символов в тексте")